
# TP 2.3 – Système de recommandation d'anime

Ce notebook applique les techniques de traitement automatique du langage naturel (NLP) pour recommander des séries animées en se basant sur le synopsis. L’étude de cas suit les instructions du TP 2.3 : sélectionner un jeu de données, nettoyer les textes, les vectoriser puis utiliser un modèle adapté pour répondre à un besoin d’analyse. Le jeu de données **Anime.csv** contient plus de 21 000 titres provenant de MyAnimeList avec des informations telles que le titre et le synopsis.

L'objectif : proposer, à partir d’un ou plusieurs anime favoris, une liste d'autres anime susceptibles de plaire par similarité de synopsis.



## Chargement des données

Le fichier `Anime.csv` est chargé avec pandas. On affiche les premières lignes pour se familiariser avec la structure du jeu de données et on note qu’il contient une colonne `Synopsis` qui sera utilisée pour la comparaison textuelle.


In [ ]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Définition du chemin du fichier CSV
file_path = '/home/oai/share/Anime.csv'


In [ ]:

# Chargement du jeu de données
anime_df = pd.read_csv(file_path)

# Affichage de quelques informations pour compréhension
print(f"Nombre de lignes et de colonnes : {anime_df.shape}")
print("Colonnes disponibles :", list(anime_df.columns))

# Afficher les 3 premières lignes
anime_df.head(3)



## Prétraitement du synopsis

Conformément au processus de NLP rappelé dans la documentation du TP (suppression des caractères inutiles, mise en minuscules et suppression des mots vides【39047747134237†L115-L124】), on commence par :

* Remplacer les valeurs manquantes de la colonne `Synopsis` par des chaînes vides.
* Utiliser l’option `stop_words='english'` de `TfidfVectorizer` pour filtrer les mots très fréquents comme « the », « and », etc., qui apportent peu d'information.

La tokenisation et la vectorisation sont réalisées via `TfidfVectorizer`, qui convertit chaque synopsis en un vecteur de caractéristiques reflétant l’importance relative des mots (TF‑IDF)【39047747134237†L141-L146】.


In [ ]:

# Remplacement des NaN par des chaînes vides
anime_df['Synopsis'] = anime_df['Synopsis'].fillna('')

# Création du vectoriseur TF‑IDF
vectorizer = TfidfVectorizer(stop_words='english')

# Apprentissage des représentations et transformation des synopsis
tfidf_matrix = vectorizer.fit_transform(anime_df['Synopsis'])

# Calcul de la matrice de similarité cosinus
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Création d'un index inversé pour retrouver rapidement l'indice d'un titre
indices = pd.Series(anime_df.index, index=anime_df['Title']).drop_duplicates()



## Fonction de recommandation et choix du modèle

Dans le cadre de ce TP, nous ne cherchons pas à prédire une étiquette mais à estimer la proximité sémantique des synopsis afin de proposer des titres similaires. Un **système de recommandation basé sur le contenu** répond à cet objectif : chaque synopsis est converti en un vecteur TF‑IDF et la **similarité cosinus** permet de mesurer la proximité entre vecteurs.

### Pourquoi cette approche ?

- **Simplicité et interprétabilité :** TF‑IDF avec similarité cosinus est une méthode classique pour comparer des documents. Elle est rapide à calculer et facile à expliquer, ce qui en fait un bon point de départ pour un projet de NLP.
- **Adapté aux données disponibles :** les synopsis sont en anglais et assez longs ; utiliser un modèle de deep learning (comme des Transformers) nécessiterait davantage de ressources et d’annotations pour offrir un gain significatif. À ce stade, le contenu suffit pour différencier les titres.
- **Pas de supervision nécessaire :** contrairement à un modèle de classification, cette approche ne requiert pas d’étiquettes (ex. genre ou note) et peut fonctionner sur n’importe quel corpus de textes descriptifs.

La fonction suivante cumule les scores de similarité cosinus pour chaque favori, élimine les favoris eux‑mêmes de la liste, trie les titres restants et renvoie les plus pertinents. Les commentaires intégrés dans le code expliquent le fonctionnement étape par étape.


In [ ]:

def recommend_anime(favorites, top_n=10):
    '''
    Retourne une liste de recommandations en se basant sur la similarité du synopsis.

    Paramètres
    ----------
    favorites : list or str
        Liste des titres d'anime favoris. Si une chaîne est fournie, elle est convertie en liste.
    top_n : int, facultatif (par défaut 10)
        Nombre de recommandations souhaitées.

    Retour
    ------
    pandas.DataFrame
        Contenant les titres recommandés et le synopsis associé.
    '''

    # Vérifier l'entrée et convertir en liste si nécessaire
    if not isinstance(favorites, list):
        favorites = [favorites]

    # Initialiser un vecteur de scores à zéro
    sim_scores = np.zeros(len(anime_df))

    # Additionner les scores pour chaque favori
    for fav in favorites:
        if fav not in indices:
            print(f"Titre '{fav}' non trouvé dans le jeu de données.")
            continue
        idx = indices[fav]
        # Ajout de la ligne de similarité cosinus correspondante
        sim_scores += cosine_sim[idx]

    # Éliminer les favoris afin de ne pas les recommander de nouveau
    fav_indices = [indices[f] for f in favorites if f in indices]
    scores = [(i, score) for i, score in enumerate(sim_scores) if i not in fav_indices]

    # Tri décroissant des scores
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Sélection des indices des meilleurs résultats
    top_indices = [i for i, _ in scores[:top_n]]

    # Renvoi d'un DataFrame avec le titre et le synopsis
    return anime_df.iloc[top_indices][['Title', 'Synopsis']]



## Exemple d'utilisation

On sélectionne trois anime populaires comme favoris (vous pouvez modifier cette liste selon vos goûts) :

* `Shingeki no Kyojin` (également connu sous le titre anglais *Attack on Titan*),
* `Sword Art Online`,
* `Naruto`.

La fonction `recommend_anime` est appelée avec ces favoris pour proposer les dix titres dont le synopsis est le plus similaire.


In [ ]:

# Liste d'anime favoris
favorites_list = ['Shingeki no Kyojin', 'Sword Art Online', 'Naruto']

# Obtention de 10 recommandations
recommendations = recommend_anime(favorites_list, top_n=10)

# Afficher uniquement les titres pour plus de lisibilité
recommendations[['Title']]



## Interprétation et conclusion

Les recommandations renvoyées sont basées uniquement sur la similarité du synopsis. On peut constater que de nombreux titres suggérés appartiennent aux mêmes franchises que les favoris (suites, films ou spin‑off). C’est dû au fait que le synopsis de ces titres est très similaire ; cette méthode ne fait pas de différence entre une suite et un titre totalement distinct.

Pour obtenir des recommandations plus diversifiées, on pourrait :

- filtrer les suites ou remakes appartenant à la même franchise ;
- utiliser des métadonnées supplémentaires (genres, studios, durée) pour pénaliser les titres trop proches ;
- combiner ce système de recommandation basé sur le contenu avec des approches collaboratives.

Malgré ces limites, ce cas pratique démontre comment mettre en œuvre un algorithme simple de recommandation d'anime via le NLP et illustre les étapes essentielles (prétraitement, vectorisation et calcul de similarité) décrites dans le TP【39047747134237†L115-L124】.
